In [2]:
from distutils.core import setup
from classes.downloader import Downloader
from classes.tracks import Track
from scripts.dataset_creator import DatasetCreator
import os
import pandas as pd
from scripts import setup
import numpy as np


d = Downloader(setup.get_spotify_username())
   
sonne = Track(d.fetch_single_track_by_name('sonne'))
sonne.load_additional_song_data(d)

related = d.get_related_artists(sonne.artist_id)
#print(d.get_artist_top_tracks(sonne.artist_id))
len(d.get_all_avaliable_genres())

Username swagnacy


126

In [ ]:
for i in range(len(related['artists'])):
    print(related['artists'][i]['name'], related['artists'][i]['id'])

In [ ]:
def get_desired_data(track):
    pp = pd.DataFrame.from_dict(track.audio_features.items()).T
    pp.columns = pp.iloc[0]
    pp = pp.drop(0)
    pp = pp[['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
                                                'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
                                                'duration_ms']]
    #p = pd.DataFrame(columns=['Name', 'Id', 'Year', 'Artist', 'Artist_id'])
    pp['Name'] = [track.name]
    pp['Id'] = [track.id]
    pp['Year'] = [track.year]
    pp['Artist'] = [track.artist_name]
    pp['Artist_id'] = [track.artist_id]

    return pp[['Name', 'Id', 'Year', 'Artist', 'Artist_id','danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
                                                'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
                                                'duration_ms']]
                                                
                                        

In [ ]:
df = pd.read_csv('datasets/full.csv').dropna()
unique_ids = list(set(df['Artist_id'].tolist()))
tmp_unique = unique_ids[:]

for i, _id in enumerate(unique_ids):
    if i < 170:
        continue
    #get related artists id
    related_ids = [rel['id'] for rel in d.get_related_artists(_id)['artists']]
    #print(related_ids)

    artist = d.get_artist(_id)
    if artist['popularity'] > 45:
        #get top 10 for id
        top = d.get_artist_top_tracks(_id)
        if top['tracks'] is None:
            continue

        for tr in top['tracks']:
            track = Track(tr)
            track.load_additional_song_data(d)

            if track.audio_features is None:
                continue
            
            df = pd.concat([df, get_desired_data(track)])

    #get top 10 for every related artist
    it = 0
    for relid in related_ids:
        if it > 5:
            break
        it+=1
        if relid not in tmp_unique:
            artist = d.get_artist(relid)
            if artist['popularity'] > 45:
                tmp_unique.append(relid)
                top = d.get_artist_top_tracks(relid)
                if top['tracks'] is None:
                    continue

                for tr in top['tracks']:
                    track = Track(tr)
                    track.load_additional_song_data(d)

                    if track.audio_features is None:
                        continue

                    df = pd.concat([df, get_desired_data(track)])
    
    print(f'{i} / {len(unique_ids)-1}')
    df.to_csv('datasets/full.csv', index=False)

df = df.reset_index(drop=True)  
df.to_csv('datasets/full_dataset.csv', index=False)
df.info()


In [ ]:
d = pd.read_csv('datasets/full.csv')
d.info()